Taking the baseline and building on it, trying to improve it.

In [1]:
# imports
import json
import gzip
import numpy as np
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MultiLabelBinarizer
from lemmagen3 import Lemmatizer
from nltk.corpus import stopwords


In [2]:
def read_json(data_path: str) -> list:
    with gzip.open(data_path, 'rt', encoding='utf-8') as f:
        return json.load(f)

In [3]:
def open_json(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        try:
            data = json.load(file)
            print("JSON is valid.")
        except json.JSONDecodeError as e:
            print(f"Invalid JSON: {e}")
    return data

In [4]:
data = open_json("data/rtvslo_train.json")

JSON is valid.


In [5]:
data[0]

{'url': 'https://www.rtvslo.si/sport/kosarka/liga-nba/v-javnost-prisel-nov-posnetek-moranta-s-pistolo/668124',
 'authors': ['T. J.'],
 'date': '2023-05-14T18:17:00',
 'title': 'V javnost prišel nov posnetek Moranta s pištolo',
 'paragraphs': ['Moštvo iz Tennesseeja je ob tem zapisalo, da je Morant suspendiran z vseh klubskih dejavnosti, do preiskave disciplinske komisije Lige NBA pa dogodka ne bo komentiralo. Video je bil sicer medtem že izbrisan. Na njem se Morant vozi na sovoznikovem sedežu, kot je videti, pa v levi roki drži pištolo. Kdaj je posnetek nastal, sicer ni jasno.',
  'Marca je bil že kaznovan, potem ko je objavil posnetek iz nočnega lokala, na njem pa ima v rokah pištolo. Posnetek je nastal v Denverju, kjer je gostoval z Grizliji. Ker pištola ni bila njegova, jo je po preiskavi lige in obljubi, da bo poiskal profesionalno pomoč, odnesel z blažjo kaznijo osmih tekem prepovedi. Komisar lige Adam Silver je tedaj njegovo dejanje označil za neodgovorno, nepremišljeno in nevarn

In [6]:
tii = data[1] # test iteration instance
datetime.strptime(tii["date"].split("T")[0], "%Y-%m-%d").strftime("%A")
tii["date"].split("T")[1].split(":")[0]
tii["url"].split("/")
# tii["topics"]
" ".join(tii["authors"])
np.sqrt(tii["n_comments"])

6.244997998398398

In [7]:
i = 0
for a in data:
    if "topics" not in a.keys():
        continue
        print(i, ":", a)
    elif a["topics"] != a["url"].split("/")[3]:
        print(i, ":", "'topics' in topic v url se ne ujemata!")
        print("'{}' :: '{}'".format(a["topics"], a["url"].split("/")[3]))
        print(a)
    i+=1

In [8]:
# sestavimo prvi del podatkov - dan v tednu, ura, avtorji, topic, subtopic
data_part1 = []

for article in data: # iteriramo cez vse clanke

    # samo datum, kot string
    date_string = article["date"].split("T")[0]
    # dan v tednu iz datuma
    day_of_week = datetime.strptime(date_string, "%Y-%m-%d").strftime("%A")

    # (samo) ura objave članka
    hour = article["date"].split("T")[1].split(":")[0]

    # se minute
    minute = round(float(article["date"].split("T")[1].split(":")[1]))
    if minute < 7.5: minute = 0
    elif minute < 22.5: minute = 15
    elif minute < 37.5: minute = 30
    elif minute < 52.5: minute = 45
    elif minute < 60: minute = 59

    # lahko tudi recimo 5-minutna resolucija (resitev je grda ampak zacasna, samo za testiranje)
    # if minute < 2.5: minute = 0
    # elif minute < 7.5: minute = 5
    # elif minute < 12.5: minute = 10
    # elif minute < 17.5: minute = 15
    # elif minute < 22.5: minute = 20
    # elif minute < 27.5: minute = 25
    # elif minute < 32.5: minute = 30
    # elif minute < 37.5: minute = 35
    # elif minute < 42.5: minute = 40
    # elif minute < 47.5: minute = 45
    # elif minute < 52.5: minute = 50
    # elif minute < 57.4: minute = 55
    # else: minute = 59


    # avtorji
    # authors = " ".join(article["authors"]) # kar zdruzimo v string, da bo encoder lahko sprejel

    #topic
    if "topics" in article.keys():
        # ce ima topic, ga kar direktno vzamemo
        topic = article["topics"] # sem preveril, da je vedno isto, kot v url... - ziher je ziher
    else:
        # ce manjka topic, ga vzamemo iz url-ja
        topic = article["url"].split("/")[3]

    #subtopic (samo prvi)
    subtopic = article["url"].split("/")[4]

    # sestavimo vrstico
    new_row = [
        day_of_week,
        hour,
        minute,
        # authors,
        topic,
        subtopic
    ]

    data_part1.append(new_row)


# in damo v one-hot encoding
onehotenc = OneHotEncoder(handle_unknown="ignore") # sem probal recimo min_frequency=2
data_part1_enc = onehotenc.fit_transform(data_part1)

In [9]:
data_part1_enc

<20981x4809 sparse matrix of type '<class 'numpy.float64'>'
	with 104905 stored elements in Compressed Sparse Row format>

In [10]:
print(data_part1_enc)

  (0, 3)	1.0
  (0, 25)	1.0
  (0, 34)	1.0
  (0, 50)	1.0
  (0, 1386)	1.0
  (1, 0)	1.0
  (1, 20)	1.0
  (1, 39)	1.0
  (1, 49)	1.0
  (1, 1291)	1.0
  (2, 2)	1.0
  (2, 17)	1.0
  (2, 35)	1.0
  (2, 47)	1.0
  (2, 248)	1.0
  (3, 3)	1.0
  (3, 26)	1.0
  (3, 33)	1.0
  (3, 50)	1.0
  (3, 2163)	1.0
  (4, 6)	1.0
  (4, 15)	1.0
  (4, 41)	1.0
  (4, 47)	1.0
  (4, 559)	1.0
  :	:
  (20976, 6)	1.0
  (20976, 24)	1.0
  (20976, 38)	1.0
  (20976, 53)	1.0
  (20976, 809)	1.0
  (20977, 6)	1.0
  (20977, 17)	1.0
  (20977, 37)	1.0
  (20977, 50)	1.0
  (20977, 3772)	1.0
  (20978, 6)	1.0
  (20978, 27)	1.0
  (20978, 31)	1.0
  (20978, 52)	1.0
  (20978, 684)	1.0
  (20979, 6)	1.0
  (20979, 23)	1.0
  (20979, 34)	1.0
  (20979, 47)	1.0
  (20979, 1327)	1.0
  (20980, 6)	1.0
  (20980, 29)	1.0
  (20980, 43)	1.0
  (20980, 50)	1.0
  (20980, 1386)	1.0


In [11]:
# avtorje posebej onehot encode-amo, ker gre za sezname več avtorjev

data_authors = []

# sestavimo tabelo s seznami avtorjev za vsak clanek
for article in data:
    new_row = article["authors"]
    data_authors.append(new_row)

# z multilabelbinarizer binariziramo sezname
mlb_authors = MultiLabelBinarizer(sparse_output=True) # izhod naj bo sparse matrika
data_authors_onehot = mlb_authors.fit_transform(data_authors)

In [12]:
data_authors

[['T. J.'],
 ['La. Da.'],
 ['Katja Šifkovič', 'Program Ars'],
 ['M. R.'],
 ['P. G.'],
 ['A. G.'],
 ['N. H.'],
 ['Andrej Karoli', 'Val 202'],
 ['M. K.'],
 ['Erna Strniša', 'Radio Slovenija'],
 ['T. K. B.'],
 ['N. H.'],
 ['G. C.'],
 ['Slavko Jerič'],
 ['G. C.'],
 ['T. J.'],
 ['T. O.', 'T. J.'],
 ['N. H.'],
 ['B. V.', 'K. S.'],
 ['G. V.', 'B. V.'],
 ['A. P. J.'],
 ['Niko Hari'],
 ['A. G.'],
 ['Tilen Jamnik'],
 ['Toni Gruden', 'Slavko Jerič'],
 ['A. G.'],
 ['B. V.'],
 ['Larisa Daugul'],
 ['G. C.'],
 ['G. C.'],
 ['Žana E. Čeh'],
 ['A. G.'],
 ['Al. Ma.'],
 ['M. R.'],
 ['To. G.', 'M. R.'],
 ['Niko Hari'],
 ['G. V.', 'B. V.'],
 ['Slavko Jerič'],
 ['G. C.'],
 ['Rok Šuligoj', 'TV Slovenija'],
 ['A. J.'],
 ['G. Pr.'],
 ['N. H.', 'Nataša Uršič', 'Prvi'],
 ['A. G.'],
 ['G. C.'],
 ['J. R.'],
 ['A. J.'],
 ['M. R.'],
 ['Kaja Sajovic'],
 ['T. K. B.'],
 ['Tilen Jamnik'],
 ['J. R.'],
 ['P. G.'],
 ['Peter Kuhar', 'Program Ars'],
 ['T. O.', 'M. R.'],
 ['Al. Ma.'],
 ['B. V.'],
 ['Irena Cunja', 'TV Slovenija

In [13]:
print(data_authors_onehot)
print(data_authors_onehot.shape)

  (0, 427)	1
  (1, 227)	1
  (2, 216)	1
  (2, 365)	1
  (3, 247)	1
  (4, 344)	1
  (5, 5)	1
  (6, 316)	1
  (7, 45)	1
  (7, 499)	1
  (8, 244)	1
  (9, 120)	1
  (9, 381)	1
  (10, 431)	1
  (11, 316)	1
  (12, 131)	1
  (13, 420)	1
  (14, 131)	1
  (15, 427)	1
  (16, 434)	1
  (16, 427)	1
  (17, 316)	1
  (18, 70)	1
  (18, 199)	1
  (19, 70)	1
  :	:
  (20958, 11)	1
  (20959, 98)	1
  (20960, 433)	1
  (20961, 433)	1
  (20962, 98)	1
  (20963, 431)	1
  (20964, 199)	1
  (20965, 431)	1
  (20966, 5)	1
  (20967, 433)	1
  (20968, 431)	1
  (20969, 5)	1
  (20970, 431)	1
  (20971, 245)	1
  (20972, 431)	1
  (20973, 98)	1
  (20974, 199)	1
  (20975, 5)	1
  (20976, 316)	1
  (20977, 99)	1
  (20977, 245)	1
  (20978, 433)	1
  (20979, 367)	1
  (20979, 563)	1
  (20980, 98)	1
(20981, 571)


In [14]:
data_authors_onehot

<20981x571 sparse matrix of type '<class 'numpy.int32'>'
	with 24298 stored elements in Compressed Sparse Row format>

In [15]:
# skupaj zlepimo prejsnje feature in na novo encode-ane avtorje
data_part1_enc = hstack([data_part1_enc, data_authors_onehot])

In [16]:
data_part1_enc

<20981x5380 sparse matrix of type '<class 'numpy.float64'>'
	with 129203 stored elements in Compressed Sparse Row format>

In [17]:
data[0]["keywords"]

['Ja Morant', 'Memphis Grizzlies', 'Liga NBA']

In [18]:
data[0]["gpt_keywords"]

['Jaja Morant',
 'košarka',
 'pištola',
 'Memphis Grizzlies',
 'NBA',
 'suspenz',
 'disciplinska komisija',
 'video',
 'Denver',
 'Grizliji',
 'Adam Silver',
 'terapevt',
 'ESPN',
 'zabijanje',
 'odgovornost',
 'incident',
 'superzvezdnik',
 'strelec',
 'podajalec',
 'končnica']

In [19]:
c = 0
i = 0
for a in data:
    if len(a["gpt_keywords"]) <= 2:
        print(i, ":", a["id"], ":", a)
        print(a["gpt_keywords"])
        print(len(a["gpt_keywords"]))
        c += 1
    i += 1
print(c)

289 : 670550 : {'url': 'https://www.rtvslo.si/sport/kosarka/drzavno-kosarkarsko-prvenstvo/helios-s-preobratom-v-zadnji-cetrtini-izenacil-serijo-proti-olimpiji/670550', 'topics': 'sport', 'authors': ['M. L.'], 'date': '2023-06-03T15:37:00', 'figures': [{'caption': 'Košarkarji Heliosa so ob odlični zadnji četrtini, ki so jo dobili s 24:16, slavili v Stožicah.', 'img': 'https://img.rtvcdn.si/_up/upload/2023/06/03/66088562.jpg', 'source': 'www.alesfevzer.com'}, {'caption': 'Matic Rebec je po izključitvi Yogija Ferrella dobro izkoristil priložnost na paretu, postal je tretji strelec Ljubljančanov.', 'img': 'https://img.rtvcdn.si/_up/upload/2023/06/03/66088561.jpg', 'source': 'www.alesfevzer.com'}], 'lead': 'Košarkarji Heliosa Sunsov so dobili drugo tekmo finala Lige Nove KBM v Stožicah. Cedevito Olimpijo so premagali z 82:81. Tretja tekma serije je na sporedu v četrtek v Domžalah.', 'paragraphs': ['V seriji na tri zmage je uvodni obračun v Domžalah dobila Olimpija z 88:84, tokrat Domžalčani

In [20]:
# enako kot za avtorje naredimo se z gpt_keywords

data_gptkeywords = []

# sestavimo tabelo s seznami gpt keywordov za vsak clanek
for article in data:
    new_row = article["gpt_keywords"]
    data_gptkeywords.append(new_row)

# z multilabelbinarizer binariziramo sezname
mlb_gptkeywords = MultiLabelBinarizer(sparse_output=True) # izhod naj bo sparse matrika
data_gptkeywords_onehot = mlb_gptkeywords.fit_transform(data_gptkeywords)

In [21]:
data_gptkeywords

[['Jaja Morant',
  'košarka',
  'pištola',
  'Memphis Grizzlies',
  'NBA',
  'suspenz',
  'disciplinska komisija',
  'video',
  'Denver',
  'Grizliji',
  'Adam Silver',
  'terapevt',
  'ESPN',
  'zabijanje',
  'odgovornost',
  'incident',
  'superzvezdnik',
  'strelec',
  'podajalec',
  'končnica'],
 ['državni zbor',
  'ocena',
  'efektivnost',
  'seja',
  'zakon',
  'predlog',
  'interpelacija',
  'referendum',
  'civilna družba',
  'koalicija',
  'ovire',
  'opozicija',
  'volivci',
  'legitimnost',
  'sodelovanje',
  'kritike',
  'vodenje',
  'javnost'],
 ['Denis&Denis',
  'roman',
  'Andrej Skubic',
  'kresnik',
  'pisatelj',
  'tehnologija',
  'umetna inteligenca',
  'avatar',
  'metaverzum',
  'introspekcija',
  'glitch',
  'banalnost',
  'moškost',
  'pisateljevanje',
  'odnosi',
  'zakon',
  'starševstvo',
  'jezik',
  'paranoja'],
 ['Benjamin Šeško',
  'avstrijsko prvenstvo',
  'Salzburg',
  'Linz',
  'Austrija',
  'Gradec',
  'Tomi Horvat',
  'Jon Gorenc Stanković',
  'LASK',

In [22]:
data_gptkeywords_onehot

<20981x111424 sparse matrix of type '<class 'numpy.int32'>'
	with 420638 stored elements in Compressed Sparse Row format>

In [23]:
# skupaj zlepimo prejsnje feature in gpt keyworde
# data_part1_enc = hstack([data_part1_enc, data_gptkeywords_onehot])

In [24]:
# enako kot za avtorje naredimo se z keywords

data_keywords = []

# sestavimo tabelo s seznami keywordov za vsak clanek
for article in data:
    new_row = article["keywords"]
    data_keywords.append(new_row)

# z multilabelbinarizer binariziramo sezname
mlb_keywords = MultiLabelBinarizer(sparse_output=True) # izhod naj bo sparse matrika
data_keywords_onehot = mlb_keywords.fit_transform(data_keywords)

data_part1_enc = hstack([data_part1_enc, data_keywords_onehot])

In [25]:
data_keywords

[['Ja Morant', 'Memphis Grizzlies', 'Liga NBA'],
 ['DZ', 'deveti sklic', 'poslanci', 'seje', 'obletnica'],
 ['Andrej E. Skubic', 'Založba Goga', 'roman'],
 ['Benjamin Šeško',
  'Salzburg',
  'Sturm Gradec',
  'Tomi Horvat',
  'Jon Gorenc Stanković'],
 ['Jurij Kraigher', 'kip', 'letalstvo'],
 ['Thomas Müller', 'Serge Gnabry', 'Mathys Tel', 'Sebastien Haller'],
 ['Izboljšanje', 'Zdravstveno stanje', 'Jamie Foxx'],
 ['evrovizija', 'komentar', 'loreen', 'joker out'],
 ['Vincent van Gogh',
  'Van Goghov muzej',
  'slikarstvo',
  'postimpresionizem',
  'razstave'],
 ['Brezplačna kosila', 'šolarji', 'DZ'],
 ['Kamen', 'Hopewell Township', 'Meteorit', 'hondrit'],
 ['Abba', 'Ring Ring', 'Album', 'Jubilejna izdaja'],
 ['Koncert', 'Slovenska vojska', 'dan', 'predstavitev', 'Stožice'],
 ['Dirka po Italiji', 'start', 'cilj', 'rožnata majica'],
 ['Pavel Rupar', 'Inštitutu 1. oktober', 'Bruselj'],
 ['Dirka po Italiji', '6. etapa', 'Neapelj', 'Roglič', 'Evenepoel'],
 ['Dirka po Italiji', '9. etapa', 'k

In [26]:
data_keywords_onehot

<20981x32972 sparse matrix of type '<class 'numpy.int32'>'
	with 82430 stored elements in Compressed Sparse Row format>

In [27]:
data_part1_enc

<20981x38352 sparse matrix of type '<class 'numpy.float64'>'
	with 211633 stored elements in Compressed Sparse Row format>

In [28]:
# sestavimo se drugi del podatkov - title in besedilo
data_part2 = []

for article in data: # iteriramo cez vse clanke

    # konkateniramo naslov, lead in vse odstavke
    new_row = (article["title"] + 
               " " + 
               article["lead"] + 
               " " + 
               " ".join(article["paragraphs"]))
    # new_row = " ".join(article["paragraphs"]) # sem probal tudi to, ker to pise na slacku ampak marko mi je na vajah rekel, da je dal tudi title

    data_part2.append(new_row)

# lematizirajmo in filtrirajmo besede...
lemmatizer = Lemmatizer("sl") # slovenski lematizator
stopwords = stopwords.words("slovene") # slovenski stopwordi

data_part2_filtered = []
for article_line in data_part2:
    tokens = article_line.split()
    lemmatized_words = [lemmatizer.lemmatize(token).lower() for token in tokens]
    filtered_words = [word for word in lemmatized_words if word not in stopwords]
    new_row = " ".join(filtered_words)
    
    data_part2_filtered.append(new_row)

data_part2 = data_part2_filtered

# in vektoriziramo
vectorizer = TfidfVectorizer()
data_part2_vect = vectorizer.fit_transform(data_part2)

In [29]:
data_part2

['javnost priti nov posnetek morant pištola družben omrežje zakrožiti nov posnetek jaj moranta, imeti odličen košarkar roka pištolo. zvezdnik memphis podoben video razburiti marca. klub preklic suspendiral. moštvo tennessee zapisalo, morant suspendiran klubski dejavnosti, preiskava disciplinski komisija lig nba dogodek komentiralo. video medtem izbrisan. n morant voziti sovoznikov sedežu, videti, lev roka držati pištolo. kdati posnetek nastal, jasno. marc kaznovan, potem objaviti posnetek nočen lokala, imeti roka pištolo. posnetek nastati denverju, kjer gostovati grizliji. pištola njegova, preiskava liga obljubi, poiskati profesionalno pomoč, odnesti blag kazen tekma prepovedi. komisar liga adam silver tedaj dejanje označiti neodgovorno, nepremišljeno nevarno. marc morant obiskovati terapevta, pogovoriti silverjem. izvrsten organizator igre, navduševati eksplozivnost silovit zabijanji, zatem espn dejal, zaveda, lahko poteza izgubi. dodati je, skušati odgovornejši, pameten izogibati sla

In [30]:
data_part2_vect

<20981x208902 sparse matrix of type '<class 'numpy.float64'>'
	with 4573561 stored elements in Compressed Sparse Row format>

In [31]:
# zdruzimo oba dela
data_model = hstack([data_part1_enc, data_part2_vect])

In [32]:
data_model

<20981x247254 sparse matrix of type '<class 'numpy.float64'>'
	with 4785194 stored elements in Compressed Sparse Row format>

In [33]:
# se y oz. stevila komentarjev
ground_truth = []

for article in data:

    # korenimo st komentarjev
    new_row = np.sqrt(article["n_comments"])

    ground_truth.append(new_row)

In [34]:
ground_truth

[8.774964387392123,
 6.244997998398398,
 1.0,
 1.4142135623730951,
 2.6457513110645907,
 2.8284271247461903,
 0.0,
 5.656854249492381,
 1.7320508075688772,
 4.358898943540674,
 4.242640687119285,
 2.23606797749979,
 8.246211251235321,
 6.244997998398398,
 11.916375287812984,
 18.841443681416774,
 26.267851073127396,
 0.0,
 18.110770276274835,
 16.0312195418814,
 5.477225575051661,
 4.123105625617661,
 3.3166247903554,
 5.0,
 13.564659966250536,
 12.569805089976535,
 3.1622776601683795,
 16.97056274847714,
 6.855654600401044,
 3.605551275463989,
 3.4641016151377544,
 6.928203230275509,
 10.63014581273465,
 1.0,
 18.49324200890693,
 6.244997998398398,
 0.0,
 5.744562646538029,
 2.449489742783178,
 7.937253933193772,
 2.0,
 3.0,
 1.0,
 3.4641016151377544,
 8.0,
 0.0,
 2.8284271247461903,
 1.4142135623730951,
 7.14142842854285,
 2.0,
 4.58257569495584,
 1.0,
 8.12403840463596,
 1.0,
 26.229754097208,
 2.23606797749979,
 5.0,
 1.4142135623730951,
 4.123105625617661,
 12.727922061357855,
 0.

In [35]:
# model
model = Ridge()
model.fit(data_model, ground_truth)

Ridge()

In [36]:
# pripravimo test set na isti nacin

test = open_json("data/rtvslo_test.json")
test_part1 = []
test_part2 = []
test_authors = []
test_gptkeywords = []
test_keywords = []

for article in test:

    # samo datum, kot string
    date_string = article["date"].split("T")[0]
    # dan v tednu iz datuma
    day_of_week = datetime.strptime(date_string, "%Y-%m-%d").strftime("%A")

    # (samo) ura objave članka
    hour = article["date"].split("T")[1].split(":")[0]

    # se minute
    minute = round(float(article["date"].split("T")[1].split(":")[1]))
    if minute < 7.5: minute = 0
    elif minute < 22.5: minute = 15
    elif minute < 37.5: minute = 30
    elif minute < 52.5: minute = 45
    elif minute < 60: minute = 59

    # avtorji
    # authors = " ".join(article["authors"]) # kar zdruzimo v string, da bo encoder lahko sprejel

    #topic
    if "topics" in article.keys():
        # ce ima topic, ga kar direktno vzamemo
        topic = article["topics"] # sem preveril, da je vedno isto, kot v url... - ziher je ziher
    else:
        # ce manjka topic, ga vzamemo iz url-ja
        topic = article["url"].split("/")[3]

    #subtopic (samo prvi)
    subtopic = article["url"].split("/")[4]

    # sestavimo vrstico
    new_row1 = [
        day_of_week,
        hour,
        minute,
        # authors,
        topic,
        subtopic
    ]

    test_part1.append(new_row1)

    # konkateniramo naslov in vse odstavke
    new_row2 = (article["title"] + 
               " " + 
               article["lead"] + 
               " " + 
               " ".join(article["paragraphs"]))

    test_part2.append(new_row2)

    new_row3 = article["authors"]
    test_authors.append(new_row3)

    new_row4 = article["gpt_keywords"]
    test_gptkeywords.append(new_row4)

    new_row5 = article["keywords"]
    test_keywords.append(new_row5)

# lematizacija in filtriranje za besede besedila
test_part2_filtered = []
for article_line in test_part2:
    tokens = article_line.split()
    lemmatized_words = [lemmatizer.lemmatize(token).lower() for token in tokens]
    filtered_words = [word for word in lemmatized_words if word not in stopwords]
    new_row = " ".join(filtered_words)
    
    test_part2_filtered.append(new_row)

test_part2 = test_part2_filtered

# uporabimo isti encoder in vectorizer in mlb-je
test_part1_enc = onehotenc.transform(test_part1)
test_part2_vect = vectorizer.transform(test_part2)
test_authors_onehot = mlb_authors.transform(test_authors)
test_gptkeywords_onehot = mlb_gptkeywords.transform(test_gptkeywords)
test_keywords_onehot = mlb_keywords.transform(test_keywords)


test = hstack([
    test_part1_enc, 
    test_authors_onehot, 
    # test_gptkeywords_onehot, 
    test_keywords_onehot,
    test_part2_vect])

JSON is valid.


C:\Users\Ziga\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_label.py:900: UserWarning: unknown class(es) ['Aleksandra Dežman', 'Anja. P. Jerič', 'David Urankar (Avtomobilnost)', 'Klemen Verlič', 'Nataša Lang', 'Robert Ferlič', 'Sanja Prelević', 'Sara Maksimovič', 'Val 202; T. O.'] will be ignored
  warnings.warn(
C:\Users\Ziga\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_label.py:900: UserWarning: unknown class(es) [' cerkveni dogodek', '- Jan Kozamernik', '1. Italia', '1. Magdeburg', '1. Radio Slovenija', '1. Salomonovi otoki', '10. RTV Slovenija', '10. opozicija', '10. puppy yoga', '11. Peking', '11. Radio Prvi', '11. Vratar', '11. vaje', '12. Eric Johansson', '12. Radio Si', '12. položaji', '12. stoletje', '12. vpliv', '13. Radio Koper', '13. Staš Skube', '13. druženje', '13. narodna enotnost', '14. Blaž Janc', '14. Radio Maribor', '14. gospodarski napredek', '15. Manaseh Sogavara', '15. PSG', '15. dnevni doseg', '16. Sedemmetrovke', 

In [37]:
test_part1_enc

<568x4809 sparse matrix of type '<class 'numpy.float64'>'
	with 2747 stored elements in Compressed Sparse Row format>

In [38]:
test_authors_onehot

<568x571 sparse matrix of type '<class 'numpy.int32'>'
	with 654 stored elements in Compressed Sparse Row format>

In [39]:
test_gptkeywords_onehot

<568x111424 sparse matrix of type '<class 'numpy.int32'>'
	with 9190 stored elements in Compressed Sparse Row format>

In [40]:
test_part2_vect

<568x208902 sparse matrix of type '<class 'numpy.float64'>'
	with 128798 stored elements in Compressed Sparse Row format>

In [41]:
test

<568x247254 sparse matrix of type '<class 'numpy.float64'>'
	with 133825 stored elements in Compressed Sparse Row format>

In [42]:
# predict
predictions = model.predict(test)

# potenciranje, da dobimo iz korenov nazaj st. komentarjev
predictions_submit = []
for p in predictions:
    # predictions_submit.append(p ** 2)
    predictions_submit.append(np.square(p))

# and save
np.savetxt('predictions/predictions.txt', predictions_submit)